In [1]:
import utils
import predict
import time as tm
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F

In [3]:
# load Data
dictSize = 225
(X, y) = utils.loadData( "train", dictSize = dictSize )
X = scipy.sparse.csr_matrix.toarray(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
df = pd.DataFrame(X)
# print (df)
df.shape
df.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16     float64
17     float64
18     float64
19     float64
20     float64
21     float64
22     float64
23     float64
24     float64
25     float64
26     float64
27     float64
28     float64
29     float64
        ...   
195    float64
196    float64
197    float64
198    float64
199    float64
200    float64
201    float64
202    float64
203    float64
204    float64
205    float64
206    float64
207    float64
208    float64
209    float64
210    float64
211    float64
212    float64
213    float64
214    float64
215    float64
216    float64
217    float64
218    float64
219    float64
220    float64
221    float64
222    float64
223    float64
224    float64
Length: 225, dtype: object

In [5]:
# class nn https://stackabuse.com/introduction-to-pytorch-for-classification/
# Model for predictions
class Model(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols + num_numerical_cols

        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i

        all_layers.append(nn.Linear(layers[-1], output_size))

        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical, x_numerical):
        embeddings = []
        for i,e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)

        x_numerical = self.batch_norm_num(x_numerical)
        x = torch.cat([x, x_numerical], 1)
        x = self.layers(x)
        return x

In [6]:
# model input

In [7]:
# training the model
# create model
model = Model(categorical_embedding_sizes, numerical_data.shape[1], 2, [200,100,50], p=0.4)


NameError: name 'categorical_embedding_sizes' is not defined

In [8]:
# loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

NameError: name 'model' is not defined

In [9]:
# train model
epochs = 300
aggregated_losses = []

for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data, numerical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')


SyntaxError: invalid syntax (<ipython-input-9-3234c9029c25>, line 12)

In [10]:
# Making predictions
with torch.no_grad():
    y_val = model(categorical_test_data, numerical_test_data)
    loss = loss_function(y_val, test_outputs)
print(f'Loss: {loss:.8f}')

SyntaxError: invalid syntax (<ipython-input-10-4776c476ee82>, line 5)

In [11]:
print(y_val[:5])
y_val = np.argmax(y_val, axis=1)
print(y_val[:5])

NameError: name 'y_val' is not defined

In [12]:
df_train = pd.DataFrame(X_train)
df_test = pd.DataFrame(X_test)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,224
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# https://discuss.pytorch.org/t/multi-class-classification/47565
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)
# add softmax layer: used for multiclass classification    

In [46]:
df_train_out = pd.DataFrame(y_train)

In [47]:
model = NeuralNet(225 , 1000, 51) 

In [48]:
# loss_function = nn.BCELoss()
loss_function = nn.CrossEntropyLoss()
model_opt = optim.SGD(model.parameters(), lr = 0.02)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()
train_set_y

tensor([ 4,  2, 12,  ...,  4,  2,  3])

In [49]:
tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)

test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [50]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [51]:
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 3.953490
Train Epoch: 1 [100/6700 (1%)]	Loss: 2.316354
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.977796
Train Epoch: 1 [300/6700 (4%)]	Loss: 4.394856
Train Epoch: 1 [400/6700 (6%)]	Loss: 4.101213
Train Epoch: 1 [500/6700 (7%)]	Loss: 1.544690
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.848627
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.895411
Train Epoch: 1 [800/6700 (12%)]	Loss: 2.228084
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.588272
Train Epoch: 1 [1000/6700 (15%)]	Loss: 2.711080
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.006372
Train Epoch: 1 [1200/6700 (18%)]	Loss: 7.274386
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.186204
Train Epoch: 1 [1400/6700 (21%)]	Loss: 1.626230
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.684482
Train Epoch: 1 [1600/6700 (24%)]	Loss: 1.412902
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.054053
Train Epoch: 1 [1800/6700 (27%)]	Loss: 1.014940
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.270737
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.395914
Train Epoch

Train Epoch: 3 [3700/6700 (55%)]	Loss: 0.125220
Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.056241
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.045132
Train Epoch: 3 [4000/6700 (60%)]	Loss: 3.699961
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.188384
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.005686
Train Epoch: 3 [4300/6700 (64%)]	Loss: 1.248386
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.001011
Train Epoch: 3 [4500/6700 (67%)]	Loss: 1.946018
Train Epoch: 3 [4600/6700 (69%)]	Loss: 2.136716
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.172383
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.416266
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.358004
Train Epoch: 3 [5000/6700 (75%)]	Loss: 1.706584
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.089530
Train Epoch: 3 [5200/6700 (78%)]	Loss: 1.245943
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.597388
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.130064
Train Epoch: 3 [5500/6700 (82%)]	Loss: 1.856217
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.838698
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2

Train Epoch: 6 [700/6700 (10%)]	Loss: 0.582296
Train Epoch: 6 [800/6700 (12%)]	Loss: 0.966950
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.062418
Train Epoch: 6 [1000/6700 (15%)]	Loss: 1.573958
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.009279
Train Epoch: 6 [1200/6700 (18%)]	Loss: 0.806674
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.015865
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.010602
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.117832
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.005056
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.001794
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.011509
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.455343
Train Epoch: 6 [2000/6700 (30%)]	Loss: 6.354751
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.028286
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.287161
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.006150
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.091081
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.049678
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.301710
Train Epoch: 6 [2700/6700 (40%)]	Loss: 0.69

Train Epoch: 8 [4500/6700 (67%)]	Loss: 0.662682
Train Epoch: 8 [4600/6700 (69%)]	Loss: 0.751705
Train Epoch: 8 [4700/6700 (70%)]	Loss: 0.040474
Train Epoch: 8 [4800/6700 (72%)]	Loss: 2.439199
Train Epoch: 8 [4900/6700 (73%)]	Loss: 0.010047
Train Epoch: 8 [5000/6700 (75%)]	Loss: 0.570984
Train Epoch: 8 [5100/6700 (76%)]	Loss: 0.005585
Train Epoch: 8 [5200/6700 (78%)]	Loss: 0.191192
Train Epoch: 8 [5300/6700 (79%)]	Loss: 0.477773
Train Epoch: 8 [5400/6700 (81%)]	Loss: 0.103659
Train Epoch: 8 [5500/6700 (82%)]	Loss: 2.352078
Train Epoch: 8 [5600/6700 (84%)]	Loss: 0.839973
Train Epoch: 8 [5700/6700 (85%)]	Loss: 3.349789
Train Epoch: 8 [5800/6700 (87%)]	Loss: 1.131496
Train Epoch: 8 [5900/6700 (88%)]	Loss: 0.381372
Train Epoch: 8 [6000/6700 (90%)]	Loss: 0.011064
Train Epoch: 8 [6100/6700 (91%)]	Loss: 0.000369
Train Epoch: 8 [6200/6700 (93%)]	Loss: 0.035070
Train Epoch: 8 [6300/6700 (94%)]	Loss: 0.014193
Train Epoch: 8 [6400/6700 (96%)]	Loss: 0.021991
Train Epoch: 8 [6500/6700 (97%)]	Loss: 0

In [19]:
epochs = 50
for epochs in range(epochs):
    model_opt.zero_grad()
    out = model(train_set_X)
    loss = loss_function(out, train_set_y)
    loss.backward()
    model_opt.step()

In [20]:
ans = model.predict(test_set_X)

/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
ans.size()

torch.Size([3300, 51])

In [54]:
ans[21]

tensor([0.0161, 0.0394, 0.0504, 0.0391, 0.0390, 0.0165, 0.0179, 0.0203, 0.0193,
        0.0203, 0.0234, 0.0196, 0.0189, 0.0192, 0.0167, 0.0183, 0.0198, 0.0179,
        0.0181, 0.0181, 0.0175, 0.0171, 0.0174, 0.0164, 0.0171, 0.0183, 0.0180,
        0.0174, 0.0164, 0.0158, 0.0170, 0.0171, 0.0169, 0.0171, 0.0177, 0.0177,
        0.0172, 0.0177, 0.0172, 0.0170, 0.0174, 0.0176, 0.0177, 0.0167, 0.0169,
        0.0169, 0.0167, 0.0180, 0.0166, 0.0174, 0.0162])

In [43]:
ans_max,ind = torch.max(ans[5,:],0)

In [44]:
ind, ans_max

(tensor(2), tensor(0.0886))

In [36]:
print(test_set_y)

tensor([3, 2, 3,  ..., 9, 4, 3])
